# Lesson 3.1 Hidden Markov Models
---

## Introduction

We view the world as a series of frames or snapshots in time, each of which contains a set of random variables where some are observable, and some are not. Hidden Markov Models or HMM's, are a machine learning technique used to make sense of sequences in time. HMM's have recently come to dominate the field of speech recognition. These models are good at taking frames that make up an input signal and feedback a result based on that model. HMM's are based on a rigourous set of mathematical theory which allowed speech researchers to build onto previous mathematical results developed in other fields. HMM's must be trained for a particular end result. In speech recognition, this could be a word or  series of words. In HMMS' these models are able to distinguish a signal with noise present, or different magnitudes, or even durations of states. This is the power available in HMM's.

This lesson will cover the algorithm and methods for HMM's and their applications for use in detecting patterns in vibrational signitures in aeromechanical testing.

## The Algorithm

### Creating the Model

Hidden Markov Models are state based models of data used to recognize patterns in the data. Given a set of states S = {s1, s2, ..., si}, the series can be observed over time. For visualization, lets look at an example. Consider the following graph representing 4 states:

![graph](./images/graph.PNG)

Given these 4 states, we can compute the transition probabilities as follows:

1. State 1 consumes 10 frames. Therefore, the outpout probability is 0.1 and the probability of remaining in step 1 is 0.9.
2. State 2 consumes 5 frames. Therefore, its transition probabilities are 0.2 and 0.8.
3. State 3 consumes 20 frames. Therefore, its transistion probabilities are 0.05 and 0.95
4. State 4 consumes 3 frames. Therefore, its transition probabilites are 0.333 and 0.667

After calculating our transisition probabilities, we can draw the following Markov Model:

![examplehmm](./images/examplehmm.PNG)

Next, we need to consider the start probabilities. There are 2 different ways to look at this. First, the model could be as shown, and the entrance point to the model is only at state 1. However, we can have probable outcomes where the starting point is in the middle of the model. We can represent this in the following way, where the entrance probabilities are uniform.

![examplehmm2](./images/examplehmm2.PNG)

### The Forward-Backward Algorithm

HMM's are a temporal probabalistic models in which the state of the process is described by a single descrete variable. The possible values of the variable are the possible states of the world. We can represent HMM's in the following way:

$X_{t}$ = A single descrete state variable  
$S$ = Number of possible states

The transition model is represented by $P(X_{t} | X_{t-1})$ as an SxS matrix T where $T_{ij} = P(X_{t}=j | X_{t-1}=i)$ where $T_{ij}$ = the probability of transition from state i to state j.

Next, we look at the evidence and observations:

$e_{t}$ = Evidence variable at time, t

$P(e_{t} | X_{t})$ for each state, i for how likely it is that state i causes evidence $e_{t}$ to appear. These probabilities are represented by an SxS diagonal matrix, $O_{t}$.

Given the above, we can compute the forward equation by simpel matrix-vector operations:

$$f_{1:t+1} = \alpha O_{t+1}T^{T}f_{1:t}$$

and the backward equation:

$$b_{k+1:t} = TO_{k+1}b_{k+2:t}$$

These equations represent the forward-backward algorithm applied to a sequence of lenght *t*. The forward-backward algorithm is an inference algorithm for HMM's which compute the posterior marginals of all hidden state variables given a sequence of observations and emissions.

### The Viterbi Trellis

The Vrterbi algorithm is a dynamic programming algorithm for finding the most likely sequence of hidden states, called the Viterbi path, that results in a sequence of observed events. The Viterbi algorithm can be built and represented as a trellis, where all possible paths are determined, and take into account the transition and emission probabilities of each state to determine the best path. Let's take our HMM model displayed previously and represent that as a Viterbi Trellis:

![VT](./images/viterbiTrellis.PNG)

Notice the filling in of the transition probabilities and that each state is a row on the trellis. The trellis displayed is a simple linear model where we must enter state 1 before 2 and state 2 before state 3, etc. However, a trellis can be built for models that have several combinational types such as state 1 to state 3, skipping state 2.

Next, we take an observation made such as the sequence: {-2,-1,-1,0,1,0,1,2,2} and compute the probability of each value being generated by a state on the trellis. For this example we will use the following:

$P(S_{1} | X_{t} = -2) = 1.0$  
$P(S_{1} | X_{t} = -1) = 0.5$  
$P(S_{2} | X_{t} = -1) = 0.5$  
$P(S_{2} | X_{t} = 0) = 0.5$  
$P(S_{3} | X_{t} = 0) = 0.5$  
$P(S_{3} | X_{t} = 1) = 0.5$  
$P(S_{4} | X_{t} = 1) = 0.5$  
$P(S_{4} | X_{t} = 2) = 1.0$  

This yields the following result with the probabilities placed in the circle of the trellis:

![VT2](./images/viterbiTrellis2.PNG)

Next, we compute the path probabilities and choose the path with the highest probability. We compute these by multiplying each state value probability by the previous transition probability, and multiplying each subsequent step. By doing this, we get the following where each step probability is shown in red and the path in green:

![VT3](./images/viterbiTrellis3.PNG)

## Visual Example: Rainy or sunny?

In this example, suppose there is a person who, for whatever reason, is unable to observe the current state of the weather. Instead, they must make inferences based on discussions made with a friend. Here, we have 2 possible hidden states as "Sunny" or "Rainy". Given a set of probabilities for the Markov Model, the person must make inferences based on the actions of their friend. These actions are "Walking", "Cleaning", and "Shopping". Please view the example code below, provide observations, prior probabilities, emission probabilities, and transition probabilities and observe how the system responds.

An example Markov Model can be seen below:

![exampleChain](./images/exampleChain.PNG)
***Note:*** *Example adapted from Wikipedia resource detailed in resources which expands on the example given in the AIMA book*

In [2]:
import numpy as np
import ipywidgets as widgets
from Example1_Widgets import *

# The Viterbi Algorithm
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    for st in states:
        V[0][st] = {"prob": start_p[st] * emit_p[st][obs[0]], "prev": None}
    # Run Viterbi when t > 0
    for t in range(1, len(obs)):
        V.append({})
        for st in states:
            max_tr_prob = max(V[t-1][prev_st]["prob"]*trans_p[prev_st][st] for prev_st in states)
            for prev_st in states:
                if V[t-1][prev_st]["prob"] * trans_p[prev_st][st] == max_tr_prob:
                    max_prob = max_tr_prob * emit_p[st][obs[t]]
                    V[t][st] = {"prob": max_prob, "prev": prev_st}
                    break
#    for line in dptable(V):
#        print line
    opt = []
    # The highest probability
    max_prob = max(value["prob"] for value in V[-1].values())
    previous = None
    # Get most probable state and its backtrack
    for st, data in V[-1].items():
        if data["prob"] == max_prob:
            opt.append(st)
            previous = st
            break
    # Follow the backtrack till the first observation
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1][previous]["prev"])
        previous = V[t + 1][previous]["prev"]
    return (opt,max_prob)
        
def RunExample1(b):
    states = ('Rainy', 'Sunny')
    observations = str(Observations.value).split(',')
    start_probability = {'Rainy': R_Start.value, 'Sunny': 1-R_Start.value}
    transition_probability = {
       'Rainy' : {'Rainy': R_Transition.value, 'Sunny': 1-R_Transition.value},
       'Sunny' : {'Rainy': S_Transition.value, 'Sunny': 1-S_Transition.value},
       }
    emission_probability = {
       'Rainy' : {'walk': R_Walk.value, 'shop': R_Shop.value, 'clean': R_Clean.value},
       'Sunny' : {'walk': S_Walk.value, 'shop': S_Shop.value, 'clean': S_Clean.value},
       }
    opt,prob = viterbi(observations,states,start_probability,transition_probability,emission_probability)
    solutionWidget.value = str(opt)
    
#Display Widgets
runButton.on_click(RunExample1)
widgets.VBox([bx1,bx2,bx3,bx4,bx5,bx6,bx7,bx8,bx9,bx10,runButton,bx11])

A Jupyter Widget

## Aeromechanical Example: Recognize Engine Order Vibrational Signiture

In aeromechanical testing, we can apply a similar process as speech recognition to build a system that can detect patterns in acquisition data. In particular, we will look at building an agent to recognize when an integral vibratory response occurs. This is done (as in speech recognition) by first placing the data in the Frequency Vs. Time domain with magnitude as the 3rd axis. This is also known as a Spectrogram or maybe a time based campbell zmod. An example of this plot can be seen below:

![zmod](./images/ZMod.png)

After we acquire or simulate several data sets, we can determine the number of frames needed and our transition probabilities. We do this by use of the EM algorithm with Gaussian Mixture Models as presented in the previous section. First, we will run EM several times on the data sets and increase or decrease our cluster count each time, We will take these and run the result clusters throught a Bayesion Info Criterion function that will determine the best cluster count to use. The cluster count will be the number of frames we need to split the data into based on our clusters. An example of this can be seen below as a 5 frame model:

![zmod2](./images/ZMod2.png)

Next, we determine our probabilities for our priors, transitions, and emissions. We do this based on time. We compute the probability of the data being in state i at time t. We can determine our probabilities as follows:

![zmod2](./images/ZMod3.png)

>  
This yields the transition probabilities of:
>  
1. State 1 (Normal State) = 0.11, 0.89
2. State 2 (EO Entrant State) = 0.125, 0.875
3. State 3 (EO Response State) = 0.11, 0.89
4. State 4 (EO Exit State) = 0.125, 0.875
5. State 5 (Normal State) = 0.125, 0.875

Finally, we build our model.

![eohmm](./images/eoHmm.PNG)

Now, when we run data through this model, the data values will be assigned to states based on their gaussian distribution. The probability of the data belonging to a state as well as the transition probabilities will factor into how the model is traversed. Also, depending on how the model was trained, the model will be able to support a wide range of variations such as length of time in a state, magnitudes, and frequency.

***Note:*** *Due to the complexity of this example, it was not feasable to create an example program. The Running of the algorithm is similar to example 1.*

## Review

## Resources